In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt  
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics
from sklearn.metrics import r2_score
import sys
sys.path.append("../../Function/")
import function
import warnings
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression


In [ ]:
df1 = pd.read_excel("../../../site_info_ver_3.3.xlsx")

In [ ]:
df = df1.copy()
df.drop( ['nox' , 'pm10', 'pm2.5','so2' ],axis=1,inplace=True)
df.reset_index(drop=True,inplace=True)
df = function.split_date(df)
df.shape

In [ ]:
# df = df.dropna(axis = 1)
dummies = pd.get_dummies(df, 'Site_Name')
print(dummies.shape)
dummies.set_index('Date',inplace=True)
dummies.sort_index(inplace=True)
print(dummies.shape)

# dummies.drop(['Model_number'] , axis = 1, inplace = True)
# dummies = dummies.astype({'Total':np.float64,'Israelis_Count':np.float64, 'Tourists_Count':np.float64})
dummies.isna().any().sum()
print(dummies.shape)

# dummies = function.remove_outliers(dummies, 'Israelis_Count')
dummies.dropna(inplace=True)
print(dummies.shape)


In [ ]:
poly_res = pd.read_excel("../POLI/poly.xlsx")
# writer = pd.ExcelWriter('../../poly.xlsx', engine='xlsxwriter')
print(dummies.shape)

In [ ]:
cols_corr = function.remove_high_corr(dummies.drop(['Total' , 'Tourists_Count' ],axis=1), 'Israelis_Count', 0.4)
dummies.drop(cols_corr,axis=1,inplace=True)
cols_corr


In [ ]:
dd = dict.fromkeys(dummies.select_dtypes(np.float64).columns, np.float32)
d = dict.fromkeys(dummies.select_dtypes(np.int64).columns, np.int32)

In [ ]:
dummies = dummies.astype(d)
dummies = dummies.astype(dd)
dummies.info()

In [ ]:
size = round(len(dummies)*.8)
dataset_train = dummies[:size]
dataset_test = dummies[size:]

targets = ['Total' , 'Israelis_Count', 'Tourists_Count' ]
X_train = dataset_train.drop(targets,axis=1)
y_train = dataset_train['Israelis_Count']

X_test = dataset_test.drop(targets,axis=1)
y_test = dataset_test['Israelis_Count']

print('train x shape:',X_train.shape)
print('train y shape:',y_train.shape)
print('test x shape:',X_test.shape)
print('test y shape:',y_test.shape)

In [ ]:
from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import Normalizer 
train_scaler = MinMaxScaler()
test_scaler = MinMaxScaler()

X_train_scaled = train_scaler.fit_transform(X_train)
y_train_scaled = np.log([y_train.values+0.01]).T
X_test_scaled = test_scaler.fit_transform(X_test)
y_test_scaled = np.log([y_test.values+0.01]).T

print('train x shape:',X_train_scaled.shape)
print('train y shape:',y_train_scaled.shape)
print('test x shape:',X_test_scaled.shape)
print('test y shape:',y_test_scaled.shape)


In [ ]:
dataset_train.info()

In [ ]:
poly = PolynomialFeatures(degree=2)
        
#fit the x variable to fit a 2rd degree polynomial value
X_poly = poly.fit_transform(X_train_scaled)
print(X_poly.shape)
poly.fit(X_poly, y_train_scaled)

pol_lin_reg = LinearRegression()
pol_lin_reg.fit(X_poly, y_train_scaled)



In [ ]:
#predict the training data
predicion_scaled = pol_lin_reg.predict(poly.fit_transform(X_test_scaled))
#predicion = y_train_scaler.inverse_transform(predicion_scaled)
predicion = np.exp(predicion_scaled)
#create a pandas series of the results

predicion = round(pd.Series(predicion.reshape(predicion.shape[0]), index=y_test.index, name='predicted_entries_train'),ndigits=2)
predicion


In [ ]:
res = pd.DataFrame(data={
    'Predicion':predicion.values,
    'Actual':y_test.values}
    ,index=y_test.index)
res[['Predicion']][res.Predicion>10000] = max(res.Actual)+1000
res[res.Predicion>10000]
res.sort_index(inplace=True)

In [ ]:
res

In [ ]:
function.plot_line(res.Predicion, res.Actual)
function.plot_residuals(res.Predicion, res.Actual,ylim=(-2000,2000))

In [ ]:
ys = [dummies.Total , dummies.Israelis_Count , dummies.Tourists_Count]


index = 0
for target in targets:
    #print(target+"\n")
    f = open("./"+target+"_general.txt" , "w")
    for i in range(1,5):
        to_drop = [x for x in targets if x != target]
        X = dummies.drop(to_drop , axis=1)
        corr_df = X.corr()
        correlated = corr_df[targets[index]].loc[(abs(corr_df[targets[index]]) >= 0.10)]
        
        correlated = correlated.drop([targets[index]]).index.tolist()
        if len(correlated) == 0:
            train_df = None
            test_df = None
            continue
        #targets = [model.Total , model.Israelis_Count , model.Tourists_Count]
        #targets_vals = ['Israelis_Count','Tourists_Count','Total']
        
        #if len(X) == 0: return None , None
        X = X[correlated]
        if len(X) == 0: 
            train_df = None
            test_df = None
            continue
        
        y = ys[index]
        #print(X)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7254)
        train_df = pd.merge(left=X_train, right=y_train, left_index=True, right_index=True)
        test_df = pd.merge(left=X_test, right=y_test, left_index=True, right_index=True)
        x_train_scaler = MinMaxScaler()
        x_test_scaler = MinMaxScaler()
        # y_train_scaler = MinMaxScaler()
        # y_test_scaler = MinMaxScaler()
        X_train_scaled = x_train_scaler.fit_transform(X_train)
        X_test_scaled = x_test_scaler.fit_transform(X_test)
        # y_train_scaled = y_train_scaler.fit_transform(pd.DataFrame(y_train))
        # y_test_scaled = y_test_scaler.fit_transform(pd.DataFrame(y_test))
        y_train_scaled = np.log(y_train + 0.01)
        y_test_scaled = np.log(y_test + 0.01)
        poly = PolynomialFeatures(degree=i)
        
        #fit the x variable to fit a 2rd degree polynomial value
        X_poly = poly.fit_transform(X_train_scaled)
        poly.fit(X_poly, y_train_scaled)
        pol_lin_reg = LinearRegression()
        
        pol_lin_reg.fit(X_poly, y_train_scaled)
        
        #predict the training data
        y_train_pred_scaled = pol_lin_reg.predict(poly.fit_transform(X_train_scaled))
        #y_train_pred = y_train_scaler.inverse_transform(y_train_pred_scaled)
        y_train_pred = np.exp(y_train_pred_scaled)
        #create a pandas series of the results
        y_train_pred = round(pd.Series(y_train_pred, index=y_train.index, name='predicted_entries_train'),ndigits=2)
        
        #Add the results to the DF
        train_df = pd.merge(left=train_df, right=y_train_pred , left_index=True, right_index=True)
        #train_df.head()
        y_test_pred_scaled = pol_lin_reg.predict(poly.fit_transform(X_test_scaled))
        y_test_pred = np.exp(y_test_pred_scaled)
        #y_test_pred = y_test_scaler.inverse_transform(y_test_pred_scaled)
        #create a pandas series of the results
        y_test_pred = round(pd.Series(y_test_pred, index=y_test.index, name='predicted_entries_test'),ndigits=2)
        #Add the results to the DF
        test_df = pd.merge(left=test_df, right=y_test_pred , left_index=True, right_index=True)
        if train_df is None and test_df is None : continue
        train_mse = metrics.mean_squared_error(train_df[targets[index]], train_df.predicted_entries_train)
        train_rmse = np.sqrt(metrics.mean_squared_error(train_df[targets[index]], train_df.predicted_entries_train))
        train_mae = metrics.mean_absolute_error(train_df[targets[index]], train_df.predicted_entries_train)
        train_std = np.std(train_df.predicted_entries_train)
        r2_train = r2_score(train_df[targets[index]], train_df.predicted_entries_train)
        f.write("Degrees: "+str(i)+",Target: "+target+"\n")
        f.write("------ TRAIN DATA ------\n")
        f.write("MSE : "+str(train_mse)+", RMSE: "+str(train_rmse)+", MAE : "+str(train_mae)+"\n")
        f.write("R2 TRAIN "+ str(r2_train)+"\n")
        f.write("TRAIN STD "+str(train_std)+"\n")
        try:
            test_mse = metrics.mean_squared_error(test_df[targets[index]], test_df.predicted_entries_test)
        except: pass
        try:
            test_rmse = np.sqrt(metrics.mean_squared_error(test_df[targets[index]], test_df.predicted_entries_test))
        except: pass
        try:
            test_mae = metrics.mean_absolute_error(test_df[targets[index]], test_df.predicted_entries_test)
        except: pass
        try:
            r2_test = r2_score(test_df[targets[index]], test_df.predicted_entries_test)
        except: pass
        
        test_std = np.std(test_df.predicted_entries_test)
        f.write("------ TEST DATA ------\n")
        f.write("MSE : "+str(test_mse)+", RMSE: "+str(test_rmse)+", MAE : "+str(test_mae)+"\n")
        f.write("R2 TEST "+ str(r2_test)+"\n")
        f.write("TEST STD "+str(test_std)+"\n")
        f.write("--------------------------------\n")
        
        coeff = pol_lin_reg.coef_
        #print(coeff)
        #intercept = pol_lin_reg.intercept_
        equation =""+ str(pol_lin_reg.intercept_)
        for idx in range(len(X.columns)):
            equation += " + "+str(round(coeff[idx],4))+" * "+str(X.columns[idx])
        new_row = {'Descripton':'general model , corr >= 0.10 ,','Degree':str(i),'Target':target,'Model_type':'POLY',
        'MAE_Training':train_mae,'MSE_Training':train_mse,
    'RMSE_Training': train_rmse,'R2_Training': r2_train,
    'MAE_Test':test_mae,'MSE_Test':test_mse,'RMSE_Test': test_rmse,'R2_Test': r2_test,'TRAIN_STD':train_std,
    'TEST_STD':test_std , 'EQUATION' : str(equation),'TEST_SIZE' : len(test_df),'TRAIN_SIZE':len(train_df)}
    
        poly_res = poly_res.append(new_row,ignore_index=True)
        #return train_df,test_df
    index += 1
    f.close()
poly_res.to_excel("../POLI/poly.xlsx",index= False)

feature selection by uni test

In [ ]:
targets = ['Total' , 'Israelis_Count', 'Tourists_Count' ]
ys = [dummies.Total , dummies.Israelis_Count , dummies.Tourists_Count]


index = 0
for target in targets:
    #print(target+"\n")
    f = open("./"+target+"_general_UNI.txt" , "w")
    to_drop = [x for x in targets if x != target]
    print(to_drop)
    for i in range(1,5):
        
        X = dummies.drop(to_drop , axis=1)
        
        y = ys[index]
        chi_selector = SelectKBest(f_regression, k=10)
        chi_selector.fit(X, y)
        chi_support = chi_selector.get_support()
        chi_feature = X.loc[:,chi_support].columns.to_list()
        if target in chi_feature : chi_feature.remove(target)
        X = X[chi_feature]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7254)
        train_df = pd.merge(left=X_train, right=y_train, left_index=True, right_index=True)
        test_df = pd.merge(left=X_test, right=y_test, left_index=True, right_index=True)
        x_train_scaler = MinMaxScaler()
        x_test_scaler = MinMaxScaler()
        # y_train_scaler = MinMaxScaler()
        # y_test_scaler = MinMaxScaler()
        X_train_scaled = x_train_scaler.fit_transform(X_train)
        X_test_scaled = x_test_scaler.fit_transform(X_test)
        # y_train_scaled = y_train_scaler.fit_transform(pd.DataFrame(y_train))
        # y_test_scaled = y_test_scaler.fit_transform(pd.DataFrame(y_test))
        y_train_scaled = np.log(y_train + 0.01)
        y_test_scaled = np.log(y_test + 0.01)
        poly = PolynomialFeatures(degree=i)
        
        #fit the x variable to fit a 2rd degree polynomial value
        X_poly = poly.fit_transform(X_train_scaled)
        poly.fit(X_poly, y_train_scaled)
        pol_lin_reg = LinearRegression()
        
        pol_lin_reg.fit(X_poly, y_train_scaled)
        
        #predict the training data
        y_train_pred_scaled = pol_lin_reg.predict(poly.fit_transform(X_train_scaled))
        #y_train_pred = y_train_scaler.inverse_transform(y_train_pred_scaled)
        y_train_pred = np.exp(y_train_pred_scaled)
        #create a pandas series of the results
        y_train_pred = round(pd.Series(y_train_pred, index=y_train.index, name='predicted_entries_train'),ndigits=2)
        
        #Add the results to the DF
        train_df = pd.merge(left=train_df, right=y_train_pred , left_index=True, right_index=True)
        print(train_df.columns)
        y_test_pred_scaled = pol_lin_reg.predict(poly.fit_transform(X_test_scaled))
        y_test_pred = np.exp(y_test_pred_scaled)
        #y_test_pred = y_test_scaler.inverse_transform(y_test_pred_scaled)
        #create a pandas series of the results
        y_test_pred = round(pd.Series(y_test_pred, index=y_test.index, name='predicted_entries_test'),ndigits=2)
        #Add the results to the DF
        test_df = pd.merge(left=test_df, right=y_test_pred , left_index=True, right_index=True)
        if train_df is None and test_df is None : continue
        train_mse = metrics.mean_squared_error(train_df[targets[index]], train_df.predicted_entries_train)
        train_rmse = np.sqrt(metrics.mean_squared_error(train_df[targets[index]], train_df.predicted_entries_train))
        train_mae = metrics.mean_absolute_error(train_df[targets[index]], train_df.predicted_entries_train)
        train_std = np.std(train_df.predicted_entries_train)
        r2_train = r2_score(train_df[targets[index]], train_df.predicted_entries_train)
        f.write("Degrees: "+str(i)+",Target: "+target+"\n")
        f.write("------ TRAIN DATA ------\n")
        f.write("MSE : "+str(train_mse)+", RMSE: "+str(train_rmse)+", MAE : "+str(train_mae)+"\n")
        f.write("R2 TRAIN "+ str(r2_train)+"\n")
        f.write("TRAIN STD "+str(train_std)+"\n")
        try:
            test_mse = metrics.mean_squared_error(test_df[targets[index]], test_df.predicted_entries_test)
        except: pass
        try:
            test_rmse = np.sqrt(metrics.mean_squared_error(test_df[targets[index]], test_df.predicted_entries_test))
        except: pass
        try:
            test_mae = metrics.mean_absolute_error(test_df[targets[index]], test_df.predicted_entries_test)
        except: pass
        try:
            r2_test = r2_score(test_df[targets[index]], test_df.predicted_entries_test)
        except: pass
        
        test_std = np.std(test_df.predicted_entries_test)
        f.write("------ TEST DATA ------\n")
        f.write("MSE : "+str(test_mse)+", RMSE: "+str(test_rmse)+", MAE : "+str(test_mae)+"\n")
        f.write("R2 TEST "+ str(r2_test)+"\n")
        f.write("TEST STD "+str(test_std)+"\n")
        f.write("--------------------------------\n")
        
        coeff = pol_lin_reg.coef_
        #print(coeff)
        intercept = pol_lin_reg.intercept_
        equation =""+ str(pol_lin_reg.intercept_)
        for idx in range(len(X.columns)):
            equation += " + "+str(round(coeff[idx],4))+" * "+str(X.columns[idx])
        new_row = {'Descripton':'general model ,Uni var feature selection, k=10, Degree: '+str(i),'Target':target,'Model_type':'POLY',
        'Degree' : str(i),'MAE_Training':train_mae,'MSE_Training':train_mse,
    'RMSE_Training': train_rmse,'R2_Training': r2_train,
    'MAE_Test':test_mae,'MSE_Test':test_mse,'RMSE_Test': test_rmse,'R2_Test': r2_test,'TRAIN_STD':train_std,
    'TEST_STD':test_std , 'EQUATION' : str(equation),'TEST_SIZE' : len(test_df),'TRAIN_SIZE':len(train_df)}
    
        poly_res = poly_res.append(new_row,ignore_index=True)
        #return train_df,test_df
    index += 1
    f.close()
poly_res.to_excel("../POLI/poly.xlsx",index= False)

with no pollutions

In [ ]:
tmp = df.copy()
tmp.columns
drops = ['Date', 'Site_Name','Model_number', 'pm10', 'pm2.5',
       'nox', 'so2', 'Tel_Aviv-Yafo_pm10',
       'Jerusalem_pm10', 'Haifa_pm10', 'Ashkelon_pm10', 'Beer-Sheva_pm10',
       'Tel_Aviv-Yafo_pm2.5', 'Jerusalem_pm2.5', 'Haifa_pm2.5',
       'Ashkelon_pm2.5', 'Beer-Sheva_pm2.5', 'Tel_Aviv-Yafo_nox',
       'Jerusalem_nox', 'Haifa_nox', 'Ashkelon_nox', 'Beer-Sheva_nox',
       'Tel_Aviv-Yafo_so2', 'Jerusalem_so2', 'Haifa_so2', 'Ashkelon_so2',
       'Beer-Sheva_so2', 'is_Site_exceeded_pm10', 'is_Site_exceeded_pm2.5',
       'is_Site_exceeded_nox', 'is_Site_exceeded_so2',
       'Tel_Aviv-Yafo_pm10_exceeded', 'Jerusalem_pm10_exceeded',
       'Haifa_pm10_exceeded', 'Ashkelon_pm10_exceeded',
       'Beer-Sheva_pm10_exceeded', 'Tel_Aviv-Yafo_pm2.5_exceeded',
       'Jerusalem_pm2.5_exceeded', 'Haifa_pm2.5_exceeded',
       'Ashkelon_pm2.5_exceeded', 'Beer-Sheva_pm2.5_exceeded',
       'Tel_Aviv-Yafo_so2_exceeded', 'Jerusalem_so2_exceeded',
       'Haifa_so2_exceeded', 'Ashkelon_so2_exceeded',
       'Beer-Sheva_so2_exceeded', 'Tel_Aviv-Yafo_nox_exceeded',
       'Jerusalem_nox_exceeded', 'Haifa_nox_exceeded', 'Ashkelon_nox_exceeded',
       'Beer-Sheva_nox_exceeded']
tmp.drop(drops , axis = 1, inplace=True)

In [ ]:
correlated_features = set()
correlation_matrix = tmp.corr()
targets = ['Total' , 'Israelis_Count', 'Tourists_Count' ]

for i in range(len(correlation_matrix .columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.4:
            colname = correlation_matrix.columns[i]
            if colname not in targets:
                correlated_features.add(colname)

In [ ]:
tmp.drop(correlated_features , axis=1,inplace=True)

In [ ]:
targets = ['Total' , 'Israelis_Count', 'Tourists_Count' ]
ys = [tmp.Total , tmp.Israelis_Count , tmp.Tourists_Count]


index = 0
for target in targets:
    #print(target+"\n")
    #f = open("./"+target+"_general_UNI.txt" , "w")
    #to_drop = [x for x in targets if x != target]
    #print(to_drop)
    for i in range(1,5):
        
        X = tmp.drop(['Total' , 'Israelis_Count', 'Tourists_Count' ] , axis=1)
        
        y = ys[index]
        # chi_selector = SelectKBest(f_regression, k=10)
        # chi_selector.fit(X, y)
        # chi_support = chi_selector.get_support()
        # chi_feature = X.loc[:,chi_support].columns.to_list()
        # if target in chi_feature : chi_feature.remove(target)
        # X = X[chi_feature]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7254)
        train_df = pd.merge(left=X_train, right=y_train, left_index=True, right_index=True)
        test_df = pd.merge(left=X_test, right=y_test, left_index=True, right_index=True)
        x_train_scaler = MinMaxScaler()
        x_test_scaler = MinMaxScaler()
        # y_train_scaler = MinMaxScaler()
        # y_test_scaler = MinMaxScaler()
        X_train_scaled = x_train_scaler.fit_transform(X_train)
        X_test_scaled = x_test_scaler.fit_transform(X_test)
        # y_train_scaled = y_train_scaler.fit_transform(pd.DataFrame(y_train))
        # y_test_scaled = y_test_scaler.fit_transform(pd.DataFrame(y_test))
        y_train_scaled = np.log(y_train + 0.01)
        y_test_scaled = np.log(y_test + 0.01)
        poly = PolynomialFeatures(degree=i)
        
        #fit the x variable to fit a 2rd degree polynomial value
        X_poly = poly.fit_transform(X_train_scaled)
        poly.fit(X_poly, y_train_scaled)
        pol_lin_reg = LinearRegression()
        
        pol_lin_reg.fit(X_poly, y_train_scaled)
        
        #predict the training data
        y_train_pred_scaled = pol_lin_reg.predict(poly.fit_transform(X_train_scaled))
        #y_train_pred = y_train_scaler.inverse_transform(y_train_pred_scaled)
        y_train_pred = np.exp(y_train_pred_scaled)
        #create a pandas series of the results
        y_train_pred = round(pd.Series(y_train_pred, index=y_train.index, name='predicted_entries_train'),ndigits=2)
        
        #Add the results to the DF
        train_df = pd.merge(left=train_df, right=y_train_pred , left_index=True, right_index=True)
        print(train_df.columns)
        y_test_pred_scaled = pol_lin_reg.predict(poly.fit_transform(X_test_scaled))
        y_test_pred = np.exp(y_test_pred_scaled)
        #y_test_pred = y_test_scaler.inverse_transform(y_test_pred_scaled)
        #create a pandas series of the results
        y_test_pred = round(pd.Series(y_test_pred, index=y_test.index, name='predicted_entries_test'),ndigits=2)
        #Add the results to the DF
        test_df = pd.merge(left=test_df, right=y_test_pred , left_index=True, right_index=True)
        if train_df is None and test_df is None : continue
        train_mse = metrics.mean_squared_error(train_df[targets[index]], train_df.predicted_entries_train)
        train_rmse = np.sqrt(metrics.mean_squared_error(train_df[targets[index]], train_df.predicted_entries_train))
        train_mae = metrics.mean_absolute_error(train_df[targets[index]], train_df.predicted_entries_train)
        train_std = np.std(train_df.predicted_entries_train)
        r2_train = r2_score(train_df[targets[index]], train_df.predicted_entries_train)
        print("Degrees: "+str(i)+",Target: "+target+"\n")
        print("------ TRAIN DATA ------\n")
        print("MSE : "+str(train_mse)+", RMSE: "+str(train_rmse)+", MAE : "+str(train_mae)+"\n")
        print("R2 TRAIN "+ str(r2_train)+"\n")
        print("TRAIN STD "+str(train_std)+"\n")
        try:
            test_mse = metrics.mean_squared_error(test_df[targets[index]], test_df.predicted_entries_test)
        except: pass
        try:
            test_rmse = np.sqrt(metrics.mean_squared_error(test_df[targets[index]], test_df.predicted_entries_test))
        except: pass
        try:
            test_mae = metrics.mean_absolute_error(test_df[targets[index]], test_df.predicted_entries_test)
        except: pass
        try:
            r2_test = r2_score(test_df[targets[index]], test_df.predicted_entries_test)
        except: pass
        
        test_std = np.std(test_df.predicted_entries_test)
        print("------ TEST DATA ------\n")
        print("MSE : "+str(test_mse)+", RMSE: "+str(test_rmse)+", MAE : "+str(test_mae)+"\n")
        print("R2 TEST "+ str(r2_test)+"\n")
        print("TEST STD "+str(test_std)+"\n")
        print("--------------------------------\n")
        
        coeff = pol_lin_reg.coef_
        #print(coeff)
        #intercept = pol_lin_reg.intercept_
        equation =""+ str(pol_lin_reg.intercept_)
        for idx in range(len(X.columns)):
            equation += " + "+str(round(coeff[idx],4))+" * "+str(X.columns[idx])
    #     new_row = {'Descripton':'general model ,Uni var feature selection, k=10, Degree: '+str(i),'Target':target,'Model_type':'POLY',
    #     'Degree' : str(i),'MAE_Training':train_mae,'MSE_Training':train_mse,
    # 'RMSE_Training': train_rmse,'R2_Training': r2_train,
    # 'MAE_Test':test_mae,'MSE_Test':test_mse,'RMSE_Test': test_rmse,'R2_Test': r2_test,'TRAIN_STD':train_std,
    # 'TEST_STD':test_std , 'EQUATION' : str(equation),'TEST_SIZE' : len(test_df),'TRAIN_SIZE':len(train_df)}
    
        #poly_res = poly_res.append(new_row,ignore_index=True)
        #return train_df,test_df
    index += 1
    #f.close()
#poly_res.to_excel("../POLI/poly.xlsx",index= False)

lets try to remove the outliers

In [ ]:
def get_outliers(df, series):
  q1 = series.quantile(0.25)
  q3 = series.quantile(0.75)

  if q1*q3 == 0:
    iqr = abs(2*(q1+q3))
    toprange = iqr
    botrange = -toprange
  else:
    iqr = q3-q1
    toprange = q3 + iqr * 1.5
    botrange = q1 - iqr * 1.5

  outliers_top=df[series > toprange]
  outliers_bot= df[series < botrange]
  outliers = pd.concat([outliers_bot, outliers_top], axis=0)

  return (botrange, toprange, outliers)

In [ ]:
botrange, toprange, outliers_total = get_outliers(df, df.Total)
print(len(outliers_total))
botrange, toprange, outliers_israelis = get_outliers(df, df.Israelis_Count)
print(len(outliers_israelis))
botrange, toprange, outliers_tourists = get_outliers(df, df.Tourists_Count)
print(len(outliers_tourists))
index_list_drop  = set(list(outliers_total.index) + list(outliers_israelis.index) + list(outliers_tourists.index))
index_list = [i for i in df.index if i not in index_list_drop]

In [ ]:
df_no_outliers = df.loc[index_list]
df_no_outliers.reset_index(inplace=True, drop=True)
df_no_outliers

In [ ]:
df_no_outliers.drop(['Date','Model_number','Site_Name'] , axis = 1,inplace=True)

In [ ]:
targets = ['Total' , 'Israelis_Count', 'Tourists_Count' ]
ys = [df_no_outliers.Total , df_no_outliers.Israelis_Count , df_no_outliers.Tourists_Count]


index = 0
for target in targets:
    #print(target+"\n")
    #f = open("./"+target+"_general_UNI.txt" , "w")
    #to_drop = [x for x in targets if x != target]
    #print(to_drop)
    for i in range(3,4):
        
        X = df_no_outliers.drop(['Total' , 'Israelis_Count', 'Tourists_Count' ] , axis=1)
        
        y = ys[index]
        chi_selector = SelectKBest(f_regression, k=10)
        chi_selector.fit(X, y)
        chi_support = chi_selector.get_support()
        chi_feature = X.loc[:,chi_support].columns.to_list()
        if target in chi_feature : chi_feature.remove(target)
        X = X[chi_feature]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7254)
        train_df = pd.merge(left=X_train, right=y_train, left_index=True, right_index=True)
        test_df = pd.merge(left=X_test, right=y_test, left_index=True, right_index=True)
        x_train_scaler = MinMaxScaler()
        x_test_scaler = MinMaxScaler()
        # y_train_scaler = MinMaxScaler()
        # y_test_scaler = MinMaxScaler()
        X_train_scaled = x_train_scaler.fit_transform(X_train)
        X_test_scaled = x_test_scaler.fit_transform(X_test)
        # y_train_scaled = y_train_scaler.fit_transform(pd.DataFrame(y_train))
        # y_test_scaled = y_test_scaler.fit_transform(pd.DataFrame(y_test))
        y_train_scaled = np.log(y_train + 0.01)
        y_test_scaled = np.log(y_test + 0.01)
        poly = PolynomialFeatures(degree=i)
        
        #fit the x variable to fit a 2rd degree polynomial value
        X_poly = poly.fit_transform(X_train_scaled)
        poly.fit(X_poly, y_train_scaled)
        pol_lin_reg = LinearRegression()
        
        pol_lin_reg.fit(X_poly, y_train_scaled)
        
        #predict the training data
        y_train_pred_scaled = pol_lin_reg.predict(poly.fit_transform(X_train_scaled))
        #y_train_pred = y_train_scaler.inverse_transform(y_train_pred_scaled)
        y_train_pred = np.exp(y_train_pred_scaled)
        #create a pandas series of the results
        y_train_pred = round(pd.Series(y_train_pred, index=y_train.index, name='predicted_entries_train'),ndigits=2)
        
        #Add the results to the DF
        train_df = pd.merge(left=train_df, right=y_train_pred , left_index=True, right_index=True)
        #print(train_df.columns)
        y_test_pred_scaled = pol_lin_reg.predict(poly.fit_transform(X_test_scaled))
        y_test_pred = np.exp(y_test_pred_scaled)
        #y_test_pred = y_test_scaler.inverse_transform(y_test_pred_scaled)
        #create a pandas series of the results
        y_test_pred = round(pd.Series(y_test_pred, index=y_test.index, name='predicted_entries_test'),ndigits=2)
        #Add the results to the DF
        test_df = pd.merge(left=test_df, right=y_test_pred , left_index=True, right_index=True)
        if train_df is None and test_df is None : continue
        train_mse = metrics.mean_squared_error(train_df[targets[index]], train_df.predicted_entries_train)
        train_rmse = np.sqrt(metrics.mean_squared_error(train_df[targets[index]], train_df.predicted_entries_train))
        train_mae = metrics.mean_absolute_error(train_df[targets[index]], train_df.predicted_entries_train)
        train_std = np.std(train_df.predicted_entries_train)
        r2_train = r2_score(train_df[targets[index]], train_df.predicted_entries_train)
        print("Degrees: "+str(i)+",Target: "+target+"\n")
        print("------ TRAIN DATA ------\n")
        print("MSE : "+str(train_mse)+", RMSE: "+str(train_rmse)+", MAE : "+str(train_mae)+"\n")
        print("R2 TRAIN "+ str(r2_train)+"\n")
        print("TRAIN STD "+str(train_std)+"\n")
        try:
            test_mse = metrics.mean_squared_error(test_df[targets[index]], test_df.predicted_entries_test)
        except: pass
        try:
            test_rmse = np.sqrt(metrics.mean_squared_error(test_df[targets[index]], test_df.predicted_entries_test))
        except: pass
        try:
            test_mae = metrics.mean_absolute_error(test_df[targets[index]], test_df.predicted_entries_test)
        except: pass
        try:
            r2_test = r2_score(test_df[targets[index]], test_df.predicted_entries_test)
        except: pass
        
        test_std = np.std(test_df.predicted_entries_test)
        print("------ TEST DATA ------\n")
        print("MSE : "+str(test_mse)+", RMSE: "+str(test_rmse)+", MAE : "+str(test_mae)+"\n")
        print("R2 TEST "+ str(r2_test)+"\n")
        print("TEST STD "+str(test_std)+"\n")
        print("--------------------------------\n")
        
        coeff = pol_lin_reg.coef_
        #print(coeff)
        #intercept = pol_lin_reg.intercept_
        equation =""+ str(pol_lin_reg.intercept_)
        for idx in range(len(X.columns)):
            equation += " + "+str(round(coeff[idx],4))+" * "+str(X.columns[idx])
        print("EQUATION : " + equation)
    #     new_row = {'Descripton':'general model ,Uni var feature selection, k=10, Degree: '+str(i),'Target':target,'Model_type':'POLY',
    #     'Degree' : str(i),'MAE_Training':train_mae,'MSE_Training':train_mse,
    # 'RMSE_Training': train_rmse,'R2_Training': r2_train,
    # 'MAE_Test':test_mae,'MSE_Test':test_mse,'RMSE_Test': test_rmse,'R2_Test': r2_test,'TRAIN_STD':train_std,
    # 'TEST_STD':test_std , 'EQUATION' : str(equation),'TEST_SIZE' : len(test_df),'TRAIN_SIZE':len(train_df)}
    
        #poly_res = poly_res.append(new_row,ignore_index=True)
        #return train_df,test_df
    index += 1
    #f.close()
#poly_res.to_excel("../POLI/poly.xlsx",index= False)